# Note
This notebook has been run on Google Colab. If you try to run it locally and you encounter issues, please switch to Colab.

## Imports and libraries version adaptation

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install -q 'scipy<=1.2.1'  # scipy.misc.imread is deprecated in later versions

In [ ]:
import os, cv2, shutil, scipy.misc, warnings
from PIL import Image
import matplotlib.pyplot as plt
from math import sqrt 
import numpy as np
from scipy.spatial import distance
from numpy.core.fromnumeric import shape
from google.colab import drive

warnings.filterwarnings("ignore")
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Main network setting

In [ ]:
!git clone https://github.com/titu1994/Neural-Style-Transfer.git

Cloning into 'Neural-Style-Transfer'...
remote: Enumerating objects: 1411, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1411 (delta 5), reused 6 (delta 1), pack-reused 1393
Receiving objects: 100% (1411/1411), 68.18 MiB | 31.13 MiB/s, done.
Resolving deltas: 100% (824/824), done.


In [ ]:
nst_path = "Neural-Style-Transfer"

## Network type
NETWORK = 'INetwork' + '.py'

## Network parameters
# Loss Weights
CONTENT_WEIGHT = 0.025
STYLE_WEIGHT = 1.0
STYLE_SCALE = 1.0
TOTAL_VARIATION_WEIGHT = 8.5e-5
CONTENT_LOSS_TYPE = 0

# Training arguments
NUM_ITERATIONS = 10
MODEL = 'vgg19'
RESCALE_IMAGE = 'false'
MAINTAIN_ASPECT_RATIO = 'false'  # Set to false if OOM occurs

# Transfer Arguments
CONTENT_LAYER = 'conv' + '5_2'  # only change the number 5_2 to something in a similar format
INITIALIZATION_IMAGE = 'content'
POOLING_TYPE = 'max'

# Extra arguments
PRESERVE_COLOR = 'false'
MIN_IMPROVEMENT = 0.0

## Helper methods definition

In [ ]:
def crop(img, n_patches = 16):
    offset = int(sqrt(n_patches))
    width = img.size[0]
    height = img.size[1]
    patches = []
    
    for h in range(offset):
        for w in range(offset):
            area = (h * (height//offset), w * (width//offset), (h+1) * (height//offset), (w+1) * (width//offset))
            patches.append(img.crop(area))
    
    return patches

def calc_closest_val(values, checkMax=False):
    if (checkMax):
        closest = max(values)
    else:
        closest = min(values)
    
    return values.index(closest)

def compute_distances(img, patches, compute_eucl=True, compute_manhat=True, compute_vca=True, excluded_imgs=[]):
    dim = (4, 4)
    test_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    eucl_dist = []
    manhat_dist = []
    vca_dist = []
    
    for patch in patches:
        resized_img = cv2.resize(np.array(patch), dim, interpolation = cv2.INTER_AREA)
        
        # Compute distances only for images that have not been paired yet
        if img not in excluded_imgs:
            if compute_eucl:
                eucl_dist.append(distance.euclidean(test_img.flatten(), resized_img.flatten()))
            if compute_manhat:
                manhat_dist.append(distance.cityblock(test_img.flatten(), resized_img.flatten()))
            if compute_vca:
                vca_dist.append(distance.cosine(test_img.flatten(), resized_img.flatten()))
    
    return calc_closest_val(eucl_dist), calc_closest_val(manhat_dist), calc_closest_val(vca_dist, True)

def show_img(img, title=''):
  if type(img) == str:
    img = plt.imread(img)

  plt.axis('off')
  plt.title(title)
  plt.imshow(img)
  plt.show()

def transfer_style(crop_number, result_dir, content_fn, style_fn):
  RESULT_DIR = os.path.join(result_dir, "generated/")
  RESULT_PREFIX = RESULT_DIR + "%d_gen" % (crop_number)
  FINAL_IMAGE_PATH = RESULT_PREFIX + "_at_iteration_%d.png" % (NUM_ITERATIONS)

  if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

  run_nst_script(RESULT_PREFIX, content_fn, style_fn)

  return RESULT_DIR, FINAL_IMAGE_PATH

def run_nst_script(result_prefix, content_fn, style_fn):
  %%shell
  !python {nst_path}/{NETWORK} {content_fn} {style_fn} {result_prefix} \
    --image_size {IMAGE_SIZE} --content_weight {CONTENT_WEIGHT} --style_weight \
    {STYLE_WEIGHT} --style_scale {STYLE_SCALE} --total_variation_weight \
    {TOTAL_VARIATION_WEIGHT} --content_loss_type {CONTENT_LOSS_TYPE} --num_iter \
    {NUM_ITERATIONS} --model {MODEL} --rescale_image {RESCALE_IMAGE} \
    --maintain_aspect_ratio {MAINTAIN_ASPECT_RATIO} --content_layer {CONTENT_LAYER} \
    --init_image {INITIALIZATION_IMAGE} --pool_type {POOLING_TYPE} --preserve_color \
    {PRESERVE_COLOR} --min_improvement {MIN_IMPROVEMENT}

In [ ]:
def transfer_color(path, content_fn):
  COLOR_TRANSFER = 'color_transfer.py'
  COLOR_FINAL_IMAGE_PATH = path + '_%s_color.png'

  # Optional - Use Histogram matching (0 for no, 1 for yes)
  HISTOGRAM_MATCH = 0

  if HISTOGRAM_MATCH == 0:
    COLOR_FINAL_IMAGE_PATH = COLOR_FINAL_IMAGE_PATH % ('original')
  else:
    COLOR_FINAL_IMAGE_PATH = COLOR_FINAL_IMAGE_PATH % ('histogram')
    
  %%shell
  !python {nst_path}/{COLOR_TRANSFER} {content_fn} {path} --hist_match {HISTOGRAM_MATCH}

In [ ]:
def reconstruct(crops, base_dir, save_path, grid_height):
  col = 0
  reconstructed_img = None

  for c in range(col, len(crops), grid_height):
    vertical_reconstruction = plt.imread(os.path.join(base_dir, crops[c]))

    # stack vertically
    for h in range(1, grid_height):
      img = plt.imread(os.path.join(base_dir, crops[c+h]))
      vertical_reconstruction = np.vstack((vertical_reconstruction, img))

    # stack horizontally
    if reconstructed_img is None:
      reconstructed_img = vertical_reconstruction
    else:
      reconstructed_img = np.hstack((reconstructed_img, vertical_reconstruction))

  scipy.misc.imsave(save_path, reconstructed_img)

# Style transfer

In [ ]:
n_patches = 9 # it must be a perfect square

base_dir = 'drive/MyDrive/_Van_Gogh/results/'
folders = os.listdir(base_dir)

for f in folders:
  reconstr_dir = os.path.join(base_dir, f, 'reconstructed')
  if not os.path.exists(reconstr_dir):
    os.mkdir(reconstr_dir)
  else: # if directory exists, it means the image has already been processed
    continue
  
  num = f[f.index('_')+1:]

  content_mask = Image.open(os.path.join(base_dir, f, num + '_seg.png'))
  style_mask = Image.open(os.path.join(base_dir, f, num + '_style_seg.png'))
  content_img = Image.open(os.path.join(base_dir, f, num + '.jpg'))
  style_img = Image.open(os.path.join(base_dir, f, num + '_style.jpg'))
  
  IMAGE_SIZE = content_img.size[0] // int(sqrt(n_patches)) # set image size for Neural Style Transfer code

  print("Cropping image " + num)
  print()

  style_crops = crop(style_img, n_patches)
  cropped_content = crop(content_img, n_patches)
  cropped_mask = crop(content_mask, n_patches)

  for i, patch in enumerate(cropped_mask):
    print("\n{:.2f}% completed...\n".format(i*100/len(cropped_mask)))

    # retrieve most similar crop
    ed, md, vcad = compute_distances(np.array(patch), crop(style_mask, n_patches))
    most_sim_style = style_crops[ed] # we use the Euclidean Distance

    # save temporarily both the crops
    temp_content = "content.png"
    temp_style = "style.png"

    cropped_content[i].save(temp_content) # save the content's patch corresponding to the cropped mask
    most_sim_style.save(temp_style)

    # transfer the style from the most similar crop of the style image to the current content patch image
    res_dir, transferred_img = transfer_style(i, os.path.join(base_dir, f), temp_content, temp_style)

    # transfer the color
    transfer_color(transferred_img, temp_content)
  
  print("\n100.00% completed!\n")

  # reconstruct all the crops and save images
  print("Reconstructing images...")

  # reconstruct images generated at every iteration
  for i in range(1, NUM_ITERATIONS+1):
    crops = [img for img in os.listdir(res_dir) if img[img.index("iteration_")+10:].replace(".png", "") == str(i) and 'color' not in img]
    grid_height = int(sqrt(n_patches))

    reconstruct(crops, res_dir, os.path.join(reconstr_dir, num + '_transferred_iteration_%d.jpg' % (i)), grid_height)

  # reconstruct image with color transfer
  crops = [img for img in os.listdir(res_dir) if 'color' in img]
  reconstruct(crops, res_dir, os.path.join(reconstr_dir, num + '_color_transfer.jpg'), grid_height)

  # remove directory with partial results
  shutil.rmtree(res_dir)

  print()
  print("Done!")
  print()
  print()

Cropping image 00440_6


0.00% completed...

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

2022-01-05 17:45:05.982495: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-01-05 17:45:06.004802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-05 17:45:06.005641: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2022-01-05 17:45:06.006013: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-01-05 17:45:06.008345: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcubla

# Results visualization

In [ ]:
for f in folders:
  num = f[f.index('_')+1:]
  reconstr_dir = os.path.join(base_dir, f, 'reconstructed')

  content_img = Image.open(os.path.join(base_dir, f, num + '.jpg'))
  style_img = Image.open(os.path.join(base_dir, f, num + '_style.jpg'))
  result_no_color = Image.open(os.path.join(reconstr_dir, num + '_transferred_iteration_%d.jpg' % (NUM_ITERATIONS)))
  result_color = Image.open(os.path.join(reconstr_dir, num + '_color_transfer.jpg'))

  print("Image no. " + num)
  print("Content:")
  show_img(content_img)
  print("Style:")
  show_img(style_img)
  print("Result (no color transfer):")
  show_img(result_no_color)
  print("Result (with color transfer):")
  show_img(result_color)
  print()

Output hidden; open in https://colab.research.google.com to view.